In [1]:
from dingo import MetabolicNetwork
import numpy as np

import sys
sys.path.insert(0, '../scripts/')
from load_modify_sample_utils import load_model, get_objective_functions, get_reaction_bounds, modify_model, sample_gapsplit, plot_grid_95_reactions
from load_modify_sample_utils import sample_dingo, sample_optgp, sampling_statistics
from graphs_utils import construct_graph, plot_graph
from distributions_comparison_utils import significantly_altered_reactions
from loopless_utils import get_loopless_solutions_from_samples, calculate_affected_samples, set_bounds_from_loopless_solution_samples
from loopless_utils import calculate_distances_from_samples, calculate_distances_from_reactions, violin_plot_samples_distances


/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:

ec_cobra_model, ec_cobra_reactions, ec_dingo_model, ec_dingo_reactions = load_model("../ext_data/models/e_coli_core.xml")

objective_functions = get_objective_functions(ec_cobra_model)
print(objective_functions)

default_reaction_bounds = get_reaction_bounds(ec_cobra_model)
print(default_reaction_bounds)

ec_cobra_model_biomass_obj_0_opt, ec_dingo_model_biomass_obj_0_opt = modify_model(ec_cobra_model, objective_function="BIOMASS_Ecoli_core_w_GAM", optimal_percentage=0)


Set parameter Username
Set parameter LicenseID to value 2642044
Academic license - for non-commercial use only - expires 2026-03-25
['BIOMASS_Ecoli_core_w_GAM']
{'PFK': (0.0, 1000.0), 'PFL': (0.0, 1000.0), 'PGI': (-1000.0, 1000.0), 'PGK': (-1000.0, 1000.0), 'PGL': (0.0, 1000.0), 'ACALD': (-1000.0, 1000.0), 'AKGt2r': (-1000.0, 1000.0), 'PGM': (-1000.0, 1000.0), 'PIt2r': (-1000.0, 1000.0), 'ALCD2x': (-1000.0, 1000.0), 'ACALDt': (-1000.0, 1000.0), 'ACKr': (-1000.0, 1000.0), 'PPC': (0.0, 1000.0), 'ACONTa': (-1000.0, 1000.0), 'ACONTb': (-1000.0, 1000.0), 'ATPM': (8.39, 1000.0), 'PPCK': (0.0, 1000.0), 'ACt2r': (-1000.0, 1000.0), 'PPS': (0.0, 1000.0), 'ADK1': (-1000.0, 1000.0), 'AKGDH': (0.0, 1000.0), 'ATPS4r': (-1000.0, 1000.0), 'PTAr': (-1000.0, 1000.0), 'PYK': (0.0, 1000.0), 'BIOMASS_Ecoli_core_w_GAM': (0.0, 1000.0), 'PYRt2': (-1000.0, 1000.0), 'CO2t': (-1000.0, 1000.0), 'RPE': (-1000.0, 1000.0), 'CS': (0.0, 1000.0), 'RPI': (-1000.0, 1000.0), 'SUCCt2_2': (0.0, 1000.0), 'CYTBD': (0.0, 1000.

In [3]:

samples_dingo = sample_dingo(ec_dingo_model_biomass_obj_0_opt, reaction_in_rows = True, ess=2000)
samples_optgp = sample_optgp(ec_cobra_model_biomass_obj_0_opt, n_samples = 3000, thinning = 100, reaction_in_rows = True)


Set parameter Username
Set parameter LicenseID to value 2642044
Academic license - for non-commercial use only - expires 2026-03-25
phase 1: number of correlated samples = 500, effective sample size = 22, ratio of the maximum singilar value over the minimum singular value = 1171.43
phase 2: number of correlated samples = 500, effective sample size = 164, ratio of the maximum singilar value over the minimum singular value = 1
phase 3: number of correlated samples = 2400, effective sample size = 870
phase 4: number of correlated samples = 2400, effective sample size = 946
[5]total ess 2002: number of correlated samples = 5800


Read LP format model from file /tmp/tmppsleb06s.lp
Reading time = 0.01 seconds
: 72 rows, 190 columns, 720 nonzeros


[5]maximum marginal PSRF: 1.00181


In [4]:

samples_dingo_loopless_solutions = get_loopless_solutions_from_samples(samples_dingo, ec_cobra_model)
samples_optgp_loopless_solutions = get_loopless_solutions_from_samples(samples_optgp, ec_cobra_model)


/home/touliopoulos/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [9]:


distances_array = calculate_distances_from_reactions(samples_dingo, samples_dingo_loopless_solutions, ec_cobra_model)
violin_plot_samples_distances(distances_array)


distances_array = calculate_distances_from_reactions(samples_optgp, samples_optgp_loopless_solutions, ec_cobra_model)
violin_plot_samples_distances(distances_array)



distances_array = calculate_distances_from_samples(samples_dingo, samples_dingo_loopless_solutions, ec_cobra_model)
violin_plot_samples_distances(distances_array)


distances_array = calculate_distances_from_samples(samples_optgp, samples_optgp_loopless_solutions, ec_cobra_model)
violin_plot_samples_distances(distances_array)



In [ ]:

def find_reactants_products(cobra_model):
    
    reactions_ids =  [ reaction.id for reaction in cobra_model.reactions ]
    
    reactants_list_all_reactions = []
    products_list_all_reactions = []
    reversibility_list_all_reactions = []
    
    for reaction in reactions_ids:
        reactants_list_single_reaction = []
        products_list_single_reaction = []

        reaction_information = cobra_model.reactions.get_by_id(reaction)
        
        reactants = reaction_information.reactants
        products = reaction_information.products
        
        reversibility = reaction_information.reversibility
        reversibility_list_all_reactions.append(reversibility)
        
        for reactant in reactants:
            reactant = str(reactant)
            reactants_list_single_reaction.append(reactant)
                
        for product in products:
            product = str(product)
            products_list_single_reaction.append(product)         
                
        reactants_list_all_reactions.append(reactants_list_single_reaction)
        products_list_all_reactions.append(products_list_single_reaction)
        
    return reversibility_list_all_reactions, reactants_list_all_reactions, products_list_all_reactions
        

def sharing_metabolites(cobra_model, reaction_a, reaction_b):
    
    reversibility_list_all_reactions, reactants_list_all_reactions, products_list_all_reactions = find_reactants_products(cobra_model=ec_cobra_model)
    
    reactions_ids =  [ reaction.id for reaction in cobra_model.reactions ]

    reaction_a_index = reactions_ids.index(reaction_a)
    reaction_b_index = reactions_ids.index(reaction_b)
    
    reactants_a = reactants_list_all_reactions[reaction_a_index]
    reactants_b = reactants_list_all_reactions[reaction_b_index]
    
    products_a = products_list_all_reactions[reaction_a_index]
    products_b = products_list_all_reactions[reaction_b_index]
    
    if (reversibility_list_all_reactions[reaction_a_index] == True) or (reversibility_list_all_reactions[reaction_b_index] == True):
        for reactant_a in reactants_a:
            if (reactant_a in reactants_b) or (reactant_a in products_b):
                share_metabolites = True
            else:
                share_metabolites = False
        
        # keep searching for sharing metabolites from products of A
        if share_metabolites == False:
            for product_a in products_a:
                if (product_a in reactants_b) or (product_a in products_b):
                    share_metabolites = True
                else:
                    share_metabolites = False
    
    # not reversible reactions
    else:
        for reactant_a in reactants_a:
            if (reactant_a in products_b):
                share_metabolites = True
            else:
                share_metabolites = False
                
        # keep searching for sharing metabolites from products of A
        if share_metabolites == False:
            for product_a in products_a:
                if product_a in reactants_b:
                    share_metabolites = True
                else:
                    share_metabolites = False
                    
    # return boolean variable
    return share_metabolites
    
 
def find_connected_components(unique_loops):
    
    from collections import defaultdict
    graph = defaultdict(set)

    # Build the adjacency list by iterating through each pair in the input
    for x, y in unique_loops:
        # Add y as a neighbor of x
        graph[x].add(y)
        # Add x as a neighbor of y (because the graph is undirected)
        graph[y].add(x)
            
    # Define a function to find connected components using Depth-First Search (DFS)

    # A set to keep track of visited nodes
    visited = set()
    # A list to store the connected components
    components = []

    # Helper function to perform a recursive DFS
    def dfs(node, component):
        # Mark the current node as visited
        visited.add(node)
        # Add the node to the current component
        component.append(node)
        # Visit all unvisited neighbors of the current node
        for neighbor in graph[node]:
            if neighbor not in visited:
                dfs(neighbor, component)

    # Iterate through all nodes in the graph
    for node in graph:
        # If the node has not been visited, it's the start of a new connected component
        if node not in visited:
            # Create a new component (list) to hold connected nodes
            component = []
            # Perform DFS starting from this node
            dfs(node, component)
            # Add the completed component to the list of components
            components.append(component)

    # Return the list of connected components that correspond to loops
    loops = components
    
    return loops  

            
            
def identify_loops(distances_array, cobra_model, distances_tol=1e-3, correlation_matrix=None, correlation_tol=100):
    
    loops = []
    
    # model to be modified
    modifed_cobra_model = cobra_model.copy()
    
    indices = np.where(distances_array >= distances_tol)[0]
    
    cobra_reactions_str = [str(reaction.id) for reaction in cobra_model.reactions]

    reactions_in_loops = [ec_cobra_reactions[index] for index in indices]
    reactions_in_loops_ids = [reaction.id for reaction in reactions_in_loops]
    
    # indices of sorted distances (biggest to smallest)
    sorted_indices = np.argsort(distances_array)[::-1]
    
    for index_a in sorted_indices:
        if cobra_reactions_str[index_a] in reactions_in_loops_ids:
            #print(cobra_reactions_str[index_a])
            for index_b in sorted_indices:
                if cobra_reactions_str[index_b] in reactions_in_loops_ids:
                    if index_a != index_b:
                        reaction_a = cobra_reactions_str[index_a]
                        reaction_b = cobra_reactions_str[index_b]
                        corr_ab = correlation_matrix[index_a, index_b]
                        
                        rest_reactions = [reaction_c for reaction_c in range(correlation_matrix.shape[0]) if reaction_c not in (index_a, index_b)]
                        
                        avg_corr_a = np.mean([correlation_matrix[index_a, index_c] for index_c in rest_reactions])
                        avg_corr_b = np.mean([correlation_matrix[index_b, index_c] for index_c in rest_reactions])

                        relative_connection_strength = abs(corr_ab) / ( ( abs(avg_corr_a) + abs(avg_corr_b) ) / 2)
                        if abs(relative_connection_strength) > correlation_tol:
                            
                            boolean_sharing_metabolites = sharing_metabolites(cobra_model=ec_cobra_model, reaction_a="FRD7", reaction_b="SUCDi")
                            if boolean_sharing_metabolites == True:
                         
                                #print(f"RCS between {reaction_a} and {reaction_b}: {relative_connection_strength:.2f}")
                                
                                loops.append([reaction_a, reaction_b])
                                
    unique_loops = list({frozenset(pair) for pair in loops})
    unique_loops = [list(pair) for pair in unique_loops]
    
    loops = find_connected_components(unique_loops)

    return loops


    


distances_array_dingo = calculate_distances_from_reactions(samples_dingo, samples_dingo_loopless_solutions, ec_cobra_model)
distances_array_optgp = calculate_distances_from_reactions(samples_optgp, samples_optgp_loopless_solutions, ec_cobra_model)

from correlations_utils import linear_correlations_in_reactions
corr_matrix_dingo, _ = linear_correlations_in_reactions(samples_dingo, ec_cobra_model, pearson_cutoff = 0.3)
corr_matrix_optgp, _ = linear_correlations_in_reactions(samples_optgp, ec_cobra_model, pearson_cutoff = 0.3)


loops_dingo = identify_loops(distances_array_dingo, cobra_model=ec_cobra_model, distances_tol=10, correlation_matrix=corr_matrix_dingo, correlation_tol=200)
loops_optgp = identify_loops(distances_array_optgp, cobra_model=ec_cobra_model, distances_tol=10, correlation_matrix=corr_matrix_optgp, correlation_tol=200)

print(loops_dingo)
print(loops_optgp)

Read LP format model from file /tmp/tmpq1r6qec8.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros


/home/touliopoulos/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning:

invalid value encountered in divide

/home/touliopoulos/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning:

invalid value encountered in divide



Read LP format model from file /tmp/tmp4lfllhb0.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
[['G6PDH2r', 'FBA', 'TKT1', 'PGI', 'RPE', 'TPI', 'PGK', 'PGM', 'PGL', 'TKT2', 'TALA', 'GND'], ['PFK', 'FBP'], ['FRD7', 'SUCDi'], ['SUCCt2_2', 'SUCCt3'], ['FORt2', 'FORt']]
[['G6PDH2r', 'FBA', 'TKT1', 'PGI', 'RPE', 'TPI', 'PGK', 'PGM', 'PGL', 'TKT2', 'TALA', 'GND'], ['PFK', 'FBP'], ['FRD7', 'SUCDi'], ['SUCCt2_2', 'SUCCt3'], ['FORt', 'FORt2']]


In [41]:

modified_cobra_model = ec_cobra_model_biomass_obj_0_opt.copy()

#remove_reactions = ['FRD7', 'FORt2', 'SUCCt2_2']
remove_reactions = ['FRD7', 'FORt2', 'FBP', 'SUCCt3']
remove_reactions = ['FRD7']

for reaction in remove_reactions:
    modified_cobra_model.reactions.get_by_id(reaction).lower_bound = 0
    modified_cobra_model.reactions.get_by_id(reaction).upper_bound = 0
    
    
modified_dingo_model = MetabolicNetwork.from_cobra_model(modified_cobra_model)
samples_dingo_modified = sample_dingo(modified_dingo_model, reaction_in_rows = True, ess=2000)

samples_optgp_modified = sample_optgp(modified_cobra_model, n_samples = 3000, thinning = 100, reaction_in_rows = True)


Read LP format model from file /tmp/tmpw2784buc.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
phase 1: number of correlated samples = 500, effective sample size = 47, ratio of the maximum singilar value over the minimum singular value = 364.929
phase 2: number of correlated samples = 500, effective sample size = 134, ratio of the maximum singilar value over the minimum singular value = 2.41029
phase 3: number of correlated samples = 2300, effective sample size = 828
phase 4: number of correlated samples = 2300, effective sample size = 639
phase 5: number of correlated samples = 1500, effective sample size = 482
[5]total ess 2130: number of correlated samples = 7100


Read LP format model from file /tmp/tmpwgaaauyh.lp
Reading time = 0.01 seconds
: 72 rows, 190 columns, 720 nonzeros


[5]maximum marginal PSRF: 1.00118


In [42]:

samples_dingo_modified_loopless_solutions = get_loopless_solutions_from_samples(samples_dingo_modified, modified_cobra_model)

samples_optgp_modified_loopless_solutions = get_loopless_solutions_from_samples(samples_optgp_modified, modified_cobra_model)


/home/touliopoulos/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning:

Solver status is 'infeasible'.



In [43]:


distances_array = calculate_distances_from_reactions(samples_dingo_modified, samples_dingo_modified_loopless_solutions, ec_cobra_model)
violin_plot_samples_distances(distances_array)

distances_array = calculate_distances_from_reactions(samples_optgp_modified, samples_optgp_modified_loopless_solutions, ec_cobra_model)
violin_plot_samples_distances(distances_array)


distances_array = calculate_distances_from_samples(samples_dingo_modified, samples_dingo_modified_loopless_solutions, ec_cobra_model)
violin_plot_samples_distances(distances_array)

distances_array = calculate_distances_from_samples(samples_optgp_modified, samples_optgp_modified_loopless_solutions, ec_cobra_model)
violin_plot_samples_distances(distances_array)


In [40]:


correlation_matrix_condition_1, _ = linear_correlations_in_reactions(samples_dingo_modified, ec_cobra_model, pearson_cutoff = 0.3)

G_condition_1, layout_condition_1  = construct_graph(correlation_matrix_condition_1,
                cobra_model = ec_cobra_model,
                correction=True,
                remove_unconnected_nodes = True
                )
plot_graph(G_condition_1, layout_condition_1)



correlation_matrix_condition_1, _ = linear_correlations_in_reactions(samples_optgp_modified, ec_cobra_model, pearson_cutoff = 0.3)

G_condition_1, layout_condition_1  = construct_graph(correlation_matrix_condition_1,
                cobra_model = ec_cobra_model,
                correction=True,
                remove_unconnected_nodes = True
                )
plot_graph(G_condition_1, layout_condition_1)


/home/touliopoulos/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning:

invalid value encountered in divide

/home/touliopoulos/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning:

invalid value encountered in divide

